In [1]:
import pandas as pd, numpy as np
from scipy.linalg import svd
import os

# Read .trc files
folder_path = os.path.join('..', 'Vicon Data', 'Shallow Squat', 'Raw')
patched_path = os.path.join('..', 'Vicon Data', 'Shallow Squat', 'Patched')
trc_files = [f for f in os.listdir(folder_path) if f.endswith('.trc')]

dfs =[pd.read_csv(os.path.join(folder_path, trc_file), sep='\t', skiprows=4, header=0) for trc_file in trc_files]

missing = '4' #LASI (X4, Y4, Z4)
ref     = ['5', '7', '6'] #RASI (X5, Y5, Z5), RPSI (X7, Y7, Z7), LPSI (X6, Y6, Z6)

# ---------- helper ----------
def gap_ranges(mask):
    """yield (start, end) indices of each NaN run"""
    idx  = np.flatnonzero(mask)
    if not idx.size: return
    breaks = np.flatnonzero(np.diff(idx) != 1)
    starts = np.concatenate(([idx[0]], idx[breaks+1]))
    ends   = np.concatenate((idx[breaks], [idx[-1]]))
    for s, e in zip(starts, ends):
        yield s, e

In [2]:

for df, trc_file in zip(dfs, trc_files):

    print(f"Processing {trc_file}...")

    file_path = os.path.join(folder_path, trc_file)

    df.columns = ['' if col.startswith('Unnamed:') else col for col in df.columns] 
    Mbad = df['X'+missing].isna()

    # Spline interpolate small gaps in reference markers
    for m in ref:
        for ax in ['X','Y','Z']:
            df[ax+m] = df[ax+m].interpolate(method='spline', order=3, limit=5, limit_direction='both').round(4)
    
    
    Q = np.stack([np.stack([df['X'+m].values, df['Y'+m].values, df['Z'+m].values], axis=1) for m in ref], axis=-1)
    P = np.stack([df['X'+missing].values, df['Y'+missing].values, df['Z'+missing].values], axis=1)

    # ---------- fill each hole with its own local reference ----------
    for start, end in gap_ranges(Mbad):
        # 20 good frames immediately before this hole
        good = np.where(~Mbad)[0]
        before = good[good < start][-20:]   
        after  = good[good > end][:20]      
        refIdx = np.concatenate([before, after])
        Qref = Q[refIdx].mean(axis=0)
        Pref = P[refIdx].mean(axis=0)

        for fr in range(start, end+1):
            q = Q[fr]
            H = q @ Qref.T
            U,_,Vt = svd(H); R = Vt.T @ U.T
            if np.linalg.det(R) < 0: Vt[2,:]*=-1; R = Vt.T @ U.T
            t = q.mean(axis=1) - R @ Qref.mean(axis=1)
            filled = R @ Pref + t
            filled = np.round(filled, 4)      
            df.loc[fr, ['X'+missing, 'Y'+missing, 'Z'+missing]] = filled

    # ---------- write back ----------
    with open(file_path) as f: header = ''.join(f.readlines()[:4])
    patched_file_path = os.path.join(patched_path, os.path.splitext(trc_file)[0] + '_patched.trc')
    df.to_csv(patched_file_path, sep='\t', index=False)
    with open(patched_file_path,'r+') as f:
        content = f.read()
        f.seek(0); f.write(header + content)

Processing Hanif_Shallow_2_5.trc...
Processing Hanif_Shallow_5_0.trc...
Processing Hanif_Shallow_7_5.trc...
